In [22]:
from openai import OpenAI
import base64
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.images.generate(
    model="dall-e-3",
    prompt="A basketball player dunking",
    # size="1024x1024",
    response_format="b64_json",
    quality="standard",
    n=1,
)

# # Save the image to a file
# image_data = [
#     output.result
#     for output in response.output
#     if output.type == "image_generation_call"
# ]

# if image_data:
#     image_base64 = image_data[0]
#     with open("otter.png", "wb") as f:
#         f.write(base64.b64decode(image_base64))

In [2]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

models = client.models.list()
for model in models.data:
    print(model.id)

gpt-4-0613
gpt-4
gpt-3.5-turbo
o4-mini-deep-research-2025-06-26
o3-pro-2025-06-10
o4-mini-deep-research
o3-deep-research
o3-deep-research-2025-06-26
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
o1-preview-2024-09-12
o1-preview
o1-mini-2024-09-12
o1-mini
gpt-4o-realtime-preview-2024-10-01
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
o3-mini
o3-mi

In [26]:
image_b64 = response.data[0].b64_json
image_data = base64.b64decode(image_b64)

with open("output.png", "wb") as f:
    f.write(image_data)

In [12]:
from PIL import Image


def make_black_transparent(input_path: str, output_path: str, threshold: int = 40):
    """
    Reads an image and makes its dark pixels transparent.

    Args:
        input_path (str): The path to the source image.
        output_path (str): The path to save the new image with transparency.
        threshold (int): The darkness level (0-255) to qualify as "black".
                         Any pixel with R, G, and B values all below this
                         threshold will be made transparent.
    """
    try:
        img = Image.open(input_path).convert("RGBA")
        pixel_data = img.getdata()

        new_pixel_data = []
        for item in pixel_data:
            if item[0] < threshold and item[1] < threshold and item[2] < threshold:
                new_pixel_data.append((item[0], item[1], item[2], 0))
            else:
                new_pixel_data.append(item)

        img.putdata(new_pixel_data)
        img.save(output_path, "PNG")
        print(f"✅ Image with transparency saved to: {output_path}")

    except FileNotFoundError:
        print(f"❌ Error: The file at {input_path} was not found.")
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")


input_file = "image-1.png"
output_file = "resultado_final_transparente.png"
make_black_transparent(input_file, output_file, threshold=60)

✅ Image with transparency saved to: resultado_final_transparente.png


In [11]:
from PIL import Image, ImageDraw
from openai import OpenAI
import os
import base64

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
image = Image.open("image-1.png") # .convert("RGBA")
prompt = """
<ROL>
Eres un motor de IA para la edición de imágenes. Tu única función es aplicar transparencia a las áreas especificadas por una máscara.
</ROL>

<TAREA PRECISA>
1.  **Área a Modificar**: Las áreas de la imagen de referencia que corresponden a las zonas BLANCAS de la imagen de la máscara.
2.  **Acción a Realizar**: Elimina completamente el contenido de estas áreas y conviértelas en 100% transparentes (canal alfa cero).
</TAREA PRECISA>

<REGLAS DE PRESERVACIÓN INVIOLABLES>
* **Fidelidad Absoluta**: Todas las áreas de la imagen de referencia que corresponden a las zonas NEGRAS de la máscara deben permanecer PÍXEL POR PÍXEL IDÉNTICAS.
* **Sin Alteraciones**: No cambies colores, texturas, sombras, ni ningún otro atributo de las áreas no modificadas.
* **Sin Interpretación Creativa**: Tu única tarea es la eliminación precisa. No añadas, rellenes o inventes nada.
</REGLAS DE PRESERVACIÓN INVIOLABLES>

<INSTRUCCIÓN FINAL>
Ejecuta la operación de transparencia. El resultado debe ser la imagen original con agujeros transparentes donde la máscara era blanca.
</INSTRUCCIÓN FINAL>
"""
prompt_2 = "Haz que toda el área editable sea completamente transparente. El resto de la imagen debe permanecer intacto."
response = client.images.edit(
    model="gpt-image-1",
    image=open("image-1.png", "rb"),
    background="transparent",
    mask=open("mascara_generada.png", "rb"),
    prompt=prompt_2,
    size="1024x1024",
    n=1,
)
image_b64 = response.data[0].b64_json
image_data = base64.b64decode(image_b64)
with open("image-2.png", "wb") as f:
    f.write(image_data)